In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
from statsmodels.tsa.stattools import coint, adfuller
from scipy.stats import norm
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

## 📊 Data Collection: Real-World Stocks & ETFs

We'll analyze:
- **Tech Sector**: AAPL, MSFT, GOOGL, AMZN, META, NVDA
- **Financial Sector**: JPM, BAC, WFC, GS, MS, C
- **Energy Sector**: XOM, CVX, COP, SLB, EOG
- **ETFs**: SPY, QQQ, IWM, DIA, XLF, XLE, XLK
- **Precious Metals**: GLD, SLV, GDX, GDXJ

In [ ]:
# Define universe of assets
assets = {
    'Tech': ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'NFLX'],
    'Finance': ['JPM', 'BAC', 'WFC', 'GS', 'MS', 'C', 'BLK', 'SCHW'],
    'Energy': ['XOM', 'CVX', 'COP', 'SLB', 'EOG', 'PSX', 'MPC', 'VLO'],
    'Healthcare': ['JNJ', 'UNH', 'PFE', 'ABBV', 'TMO', 'MRK', 'ABT', 'LLY'],
    'ETFs': ['SPY', 'QQQ', 'IWM', 'DIA', 'XLF', 'XLE', 'XLK', 'XLV', 'XLI'],
    'Metals': ['GLD', 'SLV', 'GDX', 'GDXJ', 'PPLT', 'PALL']
}

# Flatten to single list
all_symbols = [s for sector in assets.values() for s in sector]

print(f"📈 Total assets: {len(all_symbols)}")
for sector, symbols in assets.items():
    print(f"  {sector}: {len(symbols)} symbols")

In [ ]:
# Download historical data (2 years)
end_date = datetime.now()
start_date = end_date - timedelta(days=730)

print(f"📥 Downloading data from {start_date.date()} to {end_date.date()}...")

data = yf.download(
    all_symbols,
    start=start_date,
    end=end_date,
    progress=True
)['Adj Close']

# Remove symbols with insufficient data
min_data_points = 400
data = data.dropna(axis=1, thresh=min_data_points)

print(f"\n✅ Downloaded {len(data.columns)} symbols with {len(data)} data points")
print(f"\nData shape: {data.shape}")
data.head()

## 🔬 Core Functions: Statistical Tests & OU Estimation

In [ ]:
def test_cointegration(y, x, alpha=0.05):
    """
    Engle-Granger cointegration test.
    
    Returns:
        dict: beta, adf_stat, p_value, cointegrated
    """
    # Step 1: OLS regression
    beta = np.polyfit(x, y, 1)[0]
    residuals = y - beta * x
    
    # Step 2: ADF test on residuals
    adf_result = adfuller(residuals, autolag='AIC')
    adf_stat = adf_result[0]
    p_value = adf_result[1]
    
    is_cointegrated = p_value < alpha
    
    return {
        'beta': beta,
        'adf_statistic': adf_stat,
        'p_value': p_value,
        'cointegrated': is_cointegrated,
        'residuals': residuals
    }

def estimate_ou_params(spread, dt=1/252):
    """
    Estimate Ornstein-Uhlenbeck parameters via discrete approximation.
    
    Model: X_t = mu + phi*X_{t-1} + epsilon
    
    Returns:
        dict: kappa, theta, sigma, half_life
    """
    X = spread[:-1]
    Y = spread[1:]
    
    # OLS: Y = mu + phi*X + epsilon
    mu, phi = np.polyfit(X, Y, 1)
    residuals = Y - (mu + phi * X)
    sigma_epsilon = np.std(residuals)
    
    # Convert to continuous-time parameters
    kappa = -np.log(phi) / dt if phi > 0 and phi < 1 else np.nan
    theta = mu / (1 - phi) if phi != 1 else np.nan
    sigma = sigma_epsilon * np.sqrt(-2 * np.log(phi) / dt / (1 - phi**2)) if phi > 0 and phi < 1 else np.nan
    half_life = np.log(2) / kappa if kappa > 0 else np.nan
    
    return {
        'kappa': kappa,
        'theta': theta,
        'sigma': sigma,
        'half_life': half_life
    }

def hurst_exponent(series, max_lag=20):
    """
    Calculate Hurst exponent via Rescaled Range (R/S) analysis.
    
    H < 0.5: mean-reverting
    H = 0.5: random walk
    H > 0.5: trending
    """
    lags = range(2, max_lag)
    tau = []
    rs = []
    
    for lag in lags:
        n_blocks = len(series) // lag
        if n_blocks < 2:
            continue
            
        rs_values = []
        for i in range(n_blocks):
            block = series[i*lag:(i+1)*lag]
            
            # Mean-adjusted cumulative sum
            mean_adj = block - np.mean(block)
            cum_sum = np.cumsum(mean_adj)
            
            # Range
            R = np.max(cum_sum) - np.min(cum_sum)
            
            # Standard deviation
            S = np.std(block)
            
            if S > 0 and R > 0:
                rs_values.append(R / S)
        
        if rs_values:
            tau.append(lag)
            rs.append(np.mean(rs_values))
    
    if len(tau) < 2:
        return np.nan
    
    # Linear regression: log(R/S) ~ H * log(tau)
    H = np.polyfit(np.log(tau), np.log(rs), 1)[0]
    
    return H

print("✅ Statistical functions defined")

## ⚡ HJB PDE Solver for Optimal Boundaries

### Numerical Solution via Finite Differences

Discretize the HJB equation:

$$\rho V_i = \kappa(\theta - x_i)\frac{V_{i+1} - V_{i-1}}{2\Delta x} + \frac{\sigma^2}{2}\frac{V_{i+1} - 2V_i + V_{i-1}}{(\Delta x)^2}$$

Iterate until convergence to find value function $V(x)$ and optimal boundaries.

In [ ]:
def solve_hjb_ou(kappa, theta, sigma, rho=0.04, transaction_cost=0.001, 
                 n_points=200, max_iter=2000, tol=1e-6):
    """
    Solve HJB equation for optimal switching boundaries.
    
    Args:
        kappa: mean-reversion speed
        theta: long-term mean
        sigma: volatility
        rho: discount rate
        transaction_cost: cost per trade
        n_points: grid points
        max_iter: max iterations
        tol: convergence tolerance
    
    Returns:
        dict: x grid, value function V, lower/upper boundaries
    """
    # State space: theta ± 4 standard deviations
    sigma_inf = sigma / np.sqrt(2 * kappa)
    x_min = theta - 4 * sigma_inf
    x_max = theta + 4 * sigma_inf
    x = np.linspace(x_min, x_max, n_points)
    dx = x[1] - x[0]
    
    # Initialize value function
    V = np.zeros(n_points)
    
    # Iterative solver
    for iteration in range(max_iter):
        V_old = V.copy()
        
        for i in range(1, n_points - 1):
            # Drift term
            drift = kappa * (theta - x[i]) * (V[i+1] - V[i-1]) / (2 * dx)
            
            # Diffusion term
            diffusion = 0.5 * sigma**2 * (V[i+1] - 2*V[i] + V[i-1]) / dx**2
            
            # Update
            V[i] = (drift + diffusion) / rho
        
        # Boundary conditions (Neumann)
        V[0] = V[1]
        V[-1] = V[-2]
        
        # Check convergence
        if np.max(np.abs(V - V_old)) < tol:
            break
    
    # Find optimal boundaries (where V' = ±1)
    V_prime = np.gradient(V, dx)
    
    # Lower boundary (buy signal): V' ≈ 1
    lower_half = n_points // 2
    lower_idx = np.argmin(np.abs(V_prime[:lower_half] - 1))
    lower_boundary = x[lower_idx]
    
    # Upper boundary (sell signal): V' ≈ -1
    upper_idx = lower_half + np.argmin(np.abs(V_prime[lower_half:] + 1))
    upper_boundary = x[upper_idx]
    
    return {
        'x': x,
        'V': V,
        'V_prime': V_prime,
        'lower_boundary': lower_boundary,
        'upper_boundary': upper_boundary,
        'iterations': iteration + 1
    }

print("✅ HJB solver defined")

## 🎯 Backtest Engine with Optimal Switching

In [ ]:
def backtest_optimal_switching(spread, lower_bound, upper_bound, transaction_cost=0.001):
    """
    Backtest optimal switching strategy.
    
    Rules:
    - Buy when spread < lower_bound
    - Sell when spread > upper_bound
    - Exit when spread crosses theta (zero)
    """
    position = 0  # -1 (short), 0 (flat), +1 (long)
    cash = 0
    trades = []
    pnl = []
    
    for i, z in enumerate(spread):
        current_pnl = cash + position * z
        pnl.append(current_pnl)
        
        # Entry signals
        if position == 0:
            if z < lower_bound:
                # Buy spread (expect mean-reversion up)
                position = 1
                cash -= z * (1 + transaction_cost)
                trades.append({'date': i, 'action': 'BUY', 'price': z, 'position': position})
            elif z > upper_bound:
                # Short spread (expect mean-reversion down)
                position = -1
                cash += z * (1 - transaction_cost)
                trades.append({'date': i, 'action': 'SELL', 'price': z, 'position': position})
        
        # Exit signals (cross mean)
        elif position == 1 and z > 0:
            # Close long
            cash += z * (1 - transaction_cost)
            position = 0
            trades.append({'date': i, 'action': 'CLOSE_LONG', 'price': z, 'position': position})
        
        elif position == -1 and z < 0:
            # Close short
            cash -= z * (1 + transaction_cost)
            position = 0
            trades.append({'date': i, 'action': 'CLOSE_SHORT', 'price': z, 'position': position})
    
    # Close any open position at end
    if position != 0:
        cash += position * spread.iloc[-1] * (1 - transaction_cost * np.sign(position))
        position = 0
    
    pnl = np.array(pnl)
    returns = np.diff(pnl) / (np.abs(pnl[:-1]) + 1e-10)
    
    # Calculate metrics
    total_return = pnl[-1]
    sharpe = np.mean(returns) / (np.std(returns) + 1e-10) * np.sqrt(252)
    
    # Maximum drawdown
    cummax = np.maximum.accumulate(pnl)
    drawdown = (pnl - cummax) / (cummax + 1e-10)
    max_dd = np.min(drawdown)
    
    # Win rate
    trade_pnl = [trades[i+1]['price'] - trades[i]['price'] 
                 for i in range(0, len(trades)-1, 2) if i+1 < len(trades)]
    win_rate = np.sum(np.array(trade_pnl) > 0) / len(trade_pnl) if trade_pnl else 0
    
    return {
        'total_return': total_return,
        'sharpe': sharpe,
        'max_dd': max_dd,
        'num_trades': len(trades),
        'win_rate': win_rate,
        'pnl': pnl,
        'trades': trades
    }

print("✅ Backtest engine defined")

## 🔍 Comprehensive Pair Testing Function

In [ ]:
def test_pair(symbol1, symbol2, data, significance=0.05, min_hurst=0.45):
    """
    Comprehensive pair testing pipeline.
    
    Steps:
    1. Test cointegration
    2. Calculate spread
    3. Validate mean-reversion (Hurst)
    4. Estimate OU parameters
    5. Solve HJB for optimal boundaries
    6. Backtest with optimal switching
    7. Calculate combined score
    """
    try:
        # Extract price series
        y = data[symbol1].dropna()
        x = data[symbol2].dropna()
        
        # Align
        common_idx = y.index.intersection(x.index)
        if len(common_idx) < 100:
            return None
        
        y = y.loc[common_idx].values
        x = x.loc[common_idx].values
        
        # 1. Cointegration test
        coint_result = test_cointegration(y, x, alpha=significance)
        if not coint_result['cointegrated']:
            return None
        
        # 2. Calculate spread
        beta = coint_result['beta']
        spread = pd.Series(y - beta * x)
        
        # 3. Hurst exponent
        H = hurst_exponent(spread.values)
        if np.isnan(H) or H > min_hurst:
            return None
        
        # 4. OU parameters
        ou_params = estimate_ou_params(spread)
        if np.isnan(ou_params['kappa']) or ou_params['kappa'] <= 0:
            return None
        
        # 5. HJB solver
        hjb_result = solve_hjb_ou(
            kappa=ou_params['kappa'],
            theta=ou_params['theta'],
            sigma=ou_params['sigma']
        )
        
        # 6. Backtest
        backtest_result = backtest_optimal_switching(
            spread,
            hjb_result['lower_boundary'],
            hjb_result['upper_boundary']
        )
        
        # 7. Combined score
        coint_score = 1 - coint_result['p_value']
        meanrev_score = (0.5 - H) / 0.2  # Higher for H < 0.5
        profit_score = max(0, backtest_result['total_return'])
        
        combined_score = coint_score * meanrev_score * (1 + profit_score)
        
        return {
            'pair': f"{symbol1}-{symbol2}",
            'symbol1': symbol1,
            'symbol2': symbol2,
            'beta': beta,
            'p_value': coint_result['p_value'],
            'hurst': H,
            'kappa': ou_params['kappa'],
            'theta': ou_params['theta'],
            'sigma': ou_params['sigma'],
            'half_life': ou_params['half_life'],
            'lower_boundary': hjb_result['lower_boundary'],
            'upper_boundary': hjb_result['upper_boundary'],
            'total_return': backtest_result['total_return'],
            'sharpe': backtest_result['sharpe'],
            'max_dd': backtest_result['max_dd'],
            'num_trades': backtest_result['num_trades'],
            'win_rate': backtest_result['win_rate'],
            'coint_score': coint_score,
            'meanrev_score': meanrev_score,
            'profit_score': profit_score,
            'combined_score': combined_score,
            'spread': spread,
            'pnl': backtest_result['pnl']
        }
    
    except Exception as e:
        return None

print("✅ Pair testing function defined")

## 🚀 Parallel Discovery Engine

Test all pairs in parallel using ThreadPoolExecutor for maximum speed.

In [ ]:
# Build list of pairs to test
symbols = data.columns.tolist()
n_symbols = len(symbols)

# Generate all unique pairs
pairs_to_test = []
for i in range(n_symbols):
    for j in range(i+1, n_symbols):
        pairs_to_test.append((symbols[i], symbols[j]))

print(f"🔍 Total pairs to test: {len(pairs_to_test)}")
print(f"\nExample pairs: {pairs_to_test[:5]}")

In [ ]:
# Parallel processing
from tqdm.notebook import tqdm

n_workers = 8  # Adjust based on your CPU
max_pairs = 500  # Limit for notebook demo

pairs_to_test_limited = pairs_to_test[:max_pairs]

print(f"🚀 Testing {len(pairs_to_test_limited)} pairs with {n_workers} workers...\n")

results = []

with ThreadPoolExecutor(max_workers=n_workers) as executor:
    # Submit all tasks
    futures = {
        executor.submit(test_pair, sym1, sym2, data): (sym1, sym2)
        for sym1, sym2 in pairs_to_test_limited
    }
    
    # Process results with progress bar
    for future in tqdm(as_completed(futures), total=len(futures), desc="Testing pairs"):
        result = future.result()
        if result is not None:
            results.append(result)

print(f"\n✅ Found {len(results)} cointegrated & mean-reverting pairs!")

## 📊 Results Analysis & Visualization

In [ ]:
# Convert to DataFrame
if results:
    results_df = pd.DataFrame([{
        'pair': r['pair'],
        'p_value': r['p_value'],
        'hurst': r['hurst'],
        'half_life': r['half_life'],
        'kappa': r['kappa'],
        'total_return': r['total_return'],
        'sharpe': r['sharpe'],
        'max_dd': r['max_dd'],
        'num_trades': r['num_trades'],
        'win_rate': r['win_rate'],
        'combined_score': r['combined_score']
    } for r in results])
    
    # Sort by combined score
    results_df = results_df.sort_values('combined_score', ascending=False).reset_index(drop=True)
    
    print("\n📈 Summary Statistics:\n")
    print(f"Total pairs tested: {len(pairs_to_test_limited)}")
    print(f"Cointegrated pairs: {len(results)} ({len(results)/len(pairs_to_test_limited)*100:.1f}%)")
    print(f"\nAverage Return: {results_df['total_return'].mean():.2%}")
    print(f"Average Sharpe: {results_df['sharpe'].mean():.2f}")
    print(f"Average Win Rate: {results_df['win_rate'].mean():.2%}")
    print(f"\nBest Return: {results_df['total_return'].max():.2%}")
    print(f"Best Sharpe: {results_df['sharpe'].max():.2f}")
    
    print("\n\n🏆 Top 20 Pairs by Combined Score:\n")
    display(results_df.head(20))
else:
    print("❌ No cointegrated pairs found. Try different assets or longer history.")

In [ ]:
# Visualization: Score Distribution
if results:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Combined score distribution
    axes[0, 0].hist(results_df['combined_score'], bins=30, alpha=0.7, color='blue', edgecolor='black')
    axes[0, 0].set_xlabel('Combined Score')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title('Distribution of Combined Scores')
    axes[0, 0].axvline(results_df['combined_score'].median(), color='red', linestyle='--', label='Median')
    axes[0, 0].legend()
    
    # Risk-Return scatter
    scatter = axes[0, 1].scatter(
        results_df['max_dd'],
        results_df['total_return'],
        s=results_df['num_trades'],
        c=results_df['combined_score'],
        cmap='viridis',
        alpha=0.6,
        edgecolors='black'
    )
    axes[0, 1].set_xlabel('Max Drawdown')
    axes[0, 1].set_ylabel('Total Return')
    axes[0, 1].set_title('Risk-Return Profile (size=trades, color=score)')
    plt.colorbar(scatter, ax=axes[0, 1], label='Combined Score')
    
    # Hurst distribution
    axes[1, 0].hist(results_df['hurst'], bins=20, alpha=0.7, color='green', edgecolor='black')
    axes[1, 0].set_xlabel('Hurst Exponent')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Distribution of Hurst Exponents')
    axes[1, 0].axvline(0.5, color='red', linestyle='--', label='Random Walk (H=0.5)')
    axes[1, 0].legend()
    
    # Half-life distribution
    axes[1, 1].hist(results_df['half_life'], bins=20, alpha=0.7, color='orange', edgecolor='black')
    axes[1, 1].set_xlabel('Half-Life (days)')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Distribution of Mean-Reversion Half-Lives')
    axes[1, 1].axvline(results_df['half_life'].median(), color='red', linestyle='--', label='Median')
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()

## 🎯 Deep Dive: Best Performing Pair

In [ ]:
# Analyze best pair
if results:
    best_pair = results[0]
    
    print(f"\n🏆 Best Pair: {best_pair['pair']}")
    print(f"\n📊 Statistics:")
    print(f"  Beta: {best_pair['beta']:.4f}")
    print(f"  Cointegration p-value: {best_pair['p_value']:.4f}")
    print(f"  Hurst Exponent: {best_pair['hurst']:.4f}")
    print(f"\n🌊 OU Parameters:")
    print(f"  Kappa (mean-reversion speed): {best_pair['kappa']:.4f}")
    print(f"  Theta (long-term mean): {best_pair['theta']:.4f}")
    print(f"  Sigma (volatility): {best_pair['sigma']:.4f}")
    print(f"  Half-Life: {best_pair['half_life']:.2f} days")
    print(f"\n⚡ Optimal Boundaries:")
    print(f"  Lower (buy): {best_pair['lower_boundary']:.4f}")
    print(f"  Upper (sell): {best_pair['upper_boundary']:.4f}")
    print(f"\n💰 Performance:")
    print(f"  Total Return: {best_pair['total_return']:.2%}")
    print(f"  Sharpe Ratio: {best_pair['sharpe']:.2f}")
    print(f"  Max Drawdown: {best_pair['max_dd']:.2%}")
    print(f"  Number of Trades: {best_pair['num_trades']}")
    print(f"  Win Rate: {best_pair['win_rate']:.2%}")
    print(f"\n🎯 Combined Score: {best_pair['combined_score']:.4f}")

In [ ]:
# Visualize best pair
if results:
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # Spread with boundaries
    spread = best_pair['spread']
    axes[0].plot(spread.values, label='Spread', alpha=0.7)
    axes[0].axhline(best_pair['theta'], color='black', linestyle='--', label='Mean (θ)')
    axes[0].axhline(best_pair['lower_boundary'], color='green', linestyle='--', label='Buy Signal')
    axes[0].axhline(best_pair['upper_boundary'], color='red', linestyle='--', label='Sell Signal')
    axes[0].fill_between(range(len(spread)), best_pair['lower_boundary'], 
                         best_pair['upper_boundary'], alpha=0.1, color='gray')
    axes[0].set_ylabel('Spread')
    axes[0].set_title(f'Spread Time Series: {best_pair["pair"]}')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # PnL curve
    pnl = best_pair['pnl']
    axes[1].plot(pnl, label='Cumulative PnL', color='blue', linewidth=2)
    axes[1].axhline(0, color='black', linestyle='-', linewidth=0.5)
    axes[1].fill_between(range(len(pnl)), 0, pnl, where=(pnl >= 0), 
                         color='green', alpha=0.3, label='Profit')
    axes[1].fill_between(range(len(pnl)), 0, pnl, where=(pnl < 0), 
                         color='red', alpha=0.3, label='Loss')
    axes[1].set_ylabel('PnL')
    axes[1].set_title('Strategy Performance')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    # Drawdown
    cummax = np.maximum.accumulate(pnl)
    drawdown = (pnl - cummax) / (cummax + 1e-10) * 100
    axes[2].fill_between(range(len(drawdown)), 0, drawdown, color='red', alpha=0.5)
    axes[2].set_ylabel('Drawdown (%)')
    axes[2].set_xlabel('Time')
    axes[2].set_title(f'Drawdown (Max: {best_pair["max_dd"]:.2%})')
    axes[2].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 📈 Portfolio Construction: Top N Pairs

In [ ]:
# Build portfolio from top pairs
if results:
    top_n = 10
    top_pairs = results[:top_n]
    
    print(f"\n🎯 Portfolio of Top {top_n} Pairs:\n")
    
    portfolio_metrics = {
        'avg_return': np.mean([p['total_return'] for p in top_pairs]),
        'avg_sharpe': np.mean([p['sharpe'] for p in top_pairs]),
        'avg_max_dd': np.mean([p['max_dd'] for p in top_pairs]),
        'avg_win_rate': np.mean([p['win_rate'] for p in top_pairs]),
        'total_trades': sum([p['num_trades'] for p in top_pairs])
    }
    
    print(f"Average Return: {portfolio_metrics['avg_return']:.2%}")
    print(f"Average Sharpe: {portfolio_metrics['avg_sharpe']:.2f}")
    print(f"Average Max DD: {portfolio_metrics['avg_max_dd']:.2%}")
    print(f"Average Win Rate: {portfolio_metrics['avg_win_rate']:.2%}")
    print(f"Total Trades: {portfolio_metrics['total_trades']}")
    
    # Display pairs
    portfolio_df = pd.DataFrame([{
        'Pair': p['pair'],
        'Return': f"{p['total_return']:.2%}",
        'Sharpe': f"{p['sharpe']:.2f}",
        'Max DD': f"{p['max_dd']:.2%}",
        'Win Rate': f"{p['win_rate']:.2%}",
        'Trades': p['num_trades'],
        'Score': f"{p['combined_score']:.4f}"
    } for p in top_pairs])
    
    print("\n")
    display(portfolio_df)

## 💾 Export Results

In [ ]:
# Export to CSV
if results:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'cointegration_discovery_{timestamp}.csv'
    
    results_df.to_csv(filename, index=False)
    print(f"\n✅ Results exported to: {filename}")
    print(f"   {len(results_df)} pairs saved")

## 🎓 Key Takeaways

### Statistical Validation
1. **Cointegration** ensures long-term equilibrium relationship
2. **Hurst < 0.5** confirms mean-reversion property
3. **Half-life** indicates speed of reversion (optimal: 5-20 days)

### Optimal Control
1. **HJB equation** gives theoretically optimal boundaries
2. **Viscosity solutions** handle non-smooth value functions
3. **Transaction costs** significantly impact boundaries

### Portfolio Construction
1. **Diversify** across multiple pairs for risk reduction
2. **Score ranking** balances statistical validity and profitability
3. **Sector filtering** can improve economic interpretation

### Risk Management
1. Monitor **drawdowns** continuously
2. Set **stop-losses** beyond optimal boundaries
3. Adjust **position sizes** based on volatility
4. **Walk-forward** testing for out-of-sample validation

---

## 🚀 Next Steps

1. **Real-time monitoring** with streaming data
2. **Machine learning** for pair quality prediction
3. **Multi-timeframe** analysis (daily, hourly, minute)
4. **Options strategies** for leverage and hedging
5. **Regime detection** to adapt to market conditions